In [159]:
#%pip install agentpy
#%pip install owlready2

In [160]:
from owlready2 import *
# Model design
import agentpy as ap
import numpy as np
# Visualization
import seaborn as sns

In [161]:
#Tienda

onto = get_ontology("wealth_ontology.owl").load()

#if onto is not None:
#    onto.destroy()

with onto: 
    class Agent(Thing):
        pass
    class TiendaAgente(Agent):
        pass
    class DineroEnTienda(Thing):
        pass
    class obtener_dinero(FunctionalProperty, ObjectProperty):
        domain = [TiendaAgente]
        range = [DineroEnTienda]
    class has_id(FunctionalProperty, DataProperty):
        domain = [Agent]
        range = [int]
    class dinero_value(FunctionalProperty, DataProperty):
        domain = [DineroEnTienda]
        range = [int]
    class AgenteFinanciero(Agent):
        pass
    
    class DineroFinanciero(Thing):
        pass   
    
    class has_inversion(FunctionalProperty, ObjectProperty):
        domain = [AgenteFinanciero]
        range = [DineroFinanciero]

    # Propiedad para representar la cantidad de dinero financiero (valor numérico)
    class inversion_value(FunctionalProperty, DataProperty):
        domain = [DineroFinanciero]
        range = [int]
    

In [162]:
onto.save(file="wealth_ontology.owl")

In [163]:
class Message():
    environment_buffer = []
    def __init__(self,sender=None,receiver=None,performative=None,content=None):
        self.sender = sender
        self.receiver = receiver
        self.performative = performative
        self.content = content
    def __str__(self):
        return f"\n\
        Sender: {self.sender}, \n\
        Receiver: {self.receiver}, \n\
        Performative: {self.performative}, \n\
        Content: {self.content}"
    def send(self):
        """
        The send function is used to send a message to the environment buffer.
        """
        Message.environment_buffer.append(self)


In [164]:
class TiendaAgente(ap.Agent):
    """Agente - Mariana"""
    def setup(self):
        self.recordedDinero = 0
        self.myself = onto.TiendaAgente()
        self.myself.has_id = self.id
        self.myself.has_dinero = onto.DineroEnTienda(dinero_value=0)
    
    def step(self):
        self.take_msg()
        self.recordedDinero = self.myself.has_dinero.dinero_value

    def take_msg(self):
        for msg in Message.environment_buffer:
            if msg.receiver == self.id:
                if msg.performative == "dardinero":
                    # Update own belief about wealth, using message content
                    self.myself.has_dinero.dinero_value += msg.content["dardinero"]
                    Message.environment_buffer.remove(msg)
                elif msg.performative == "darpedido": 
                    self.dar_pedido(msg.content["cantidad"])

    
    def dar_pedido(self, num):
        #TODO: cambiar sender.id a repartidor
        msg = Message(sender=self.id, receiver=0, performative="dardinero",content={"item":1, "cantidad": num})
        msg.send()
    


In [ ]:
class AgenteFinanciero(ap.Agent):
    """Agente que maneja dinero financiero, inversiones, etc."""
    def setup(self):

        self.inversionActual = 0
        
        self.myself = onto.AgenteFinanciero()
        self.myself.has_id = self.id
        
        self.myself.has_inversion = onto.DineroFinanciero(inversion_value=0)
    
    def step(self):
        """
        En cada paso, el agente revisa los mensajes y actualiza
        su conocimiento sobre el dinero financiero.
        """
        self.take_msg()
        self.inversionActual = self.myself.has_inversion.inversion_value
        
    def take_msg(self):
        
        for msg in Message.environment_buffer:
            if msg.receiver == self.id:
                
                if msg.performative == "invertir":
                    cantidad = msg.content.get("cantidad", 0)
                    self.myself.has_inversion.inversion_value += cantidad
                    print(f"AgenteFinanciero (ID {self.id}) recibe inversión de {cantidad}.")
                    
                    Message.environment_buffer.remove(msg)
                


In [165]:
class VentasModel(ap.Model):
    def setup(self):
        self.agentsTienda = ap.AgentList(self, self.p.agents, TiendaAgente)
    def step(self):
        self.agentsTienda.step()
    def update(self):
        self.record("Dinero obtenido", self.agentsTienda.recorded_Dinero)
    def end(self):
        return

In [166]:
parameters = {
    'agents': 100,
    'steps': 100
}

In [167]:
model = VentasModel(parameters)
results = model.run()

Completed: 100 steps
Run time: 0:00:00.021651
Simulation finished


In [168]:
results

DataDict {
'info': Dictionary with 9 keys
'parameters': 
    'constants': Dictionary with 2 keys
'variables': 
    'VentasModel': DataFrame with 1 variable and 101 rows
'reporters': DataFrame with 1 variable and 1 row
}